<a href="https://colab.research.google.com/github/naguzmans/opportunistic-atm/blob/main/pso_model_3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2> 3D Particle Swarm Optimization </h2>

In [10]:
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from scipy import interpolate
from tqdm import tqdm
import random
import warnings
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'

class Obstacle:
  def __init__(self, x, y, z, r):
    self.x = x
    self.y = y
    self.z = z
    self.r = r

class Particle:
  def __init__(self, origin, destination, handles):
    self.origin = origin
    self.destination = destination
    self.handles = handles

    # Define x array
    self.x, self.y, self.z = self.CreatePath(self.origin, self.destination)

    # Define velocity
    self.vx = np.zeros(handles+2)
    self.vy = np.zeros(handles+2)
    self.vz = np.zeros(handles+2)

    # Calculate Cost
    self.Cost()
    
    # Define Best
    self.Best(self.x, self.y, self.z, self.xs, self.ys, self.zs, self.cost)

  def Cost(self):

    # Create spline
    # ts = np.linspace(self.origin[0], self.destination[0], self.handles+2 )

    if self.destination[0] > self.origin[0]:
      ts = np.linspace(self.origin[0], self.destination[0], self.handles+2 )
    else:
      ts = np.linspace(self.destination[0], self.origin[0], self.handles+2 )

    spline_x = interpolate.CubicSpline(ts, self.x)
    spline_y = interpolate.CubicSpline(ts, self.y)
    spline_z = interpolate.CubicSpline(ts, self.z)

    ts2 = np.linspace(self.origin[0], self.destination[0], 100)
    self.xs = spline_x(ts2)
    self.ys = spline_y(ts2)
    self.zs = spline_z(ts2)
    # self.zs = (self.zs/self.zs.max()) * self.zlim

    dsx = spline_x.derivative(nu=1)
    dsy = spline_y.derivative(nu=1)
    dsz = spline_z.derivative(nu=1)

    self.dxs = dsx(ts2)
    self.dys = dsy(ts2)
    self.dzs = dsz(ts2)
    self.L = np.sum(np.sqrt(self.dxs**2 + self.dys**2 + self.dzs**2))
    
    violation = 0
    for i in range(0, len(obstacles)):
      distance_to_obstacle = np.sqrt((self.xs-obstacles[i].x)**2+(self.ys-obstacles[i].y)**2+(self.zs-obstacles[i].z)**2)
      v = 1-distance_to_obstacle/obstacles[i].r
      for j in range(0,len(v)):
        if v[j] < 0:
          v[j] = 0
      violation += np.mean(v)

    # Z out-of-bounds violation
    if any(i < 0 for i in self.zs):
      violation = 1e10

    # Calculate final cost
    beta = 100
    self.cost = self.L*(1+beta*violation)

  def Best(self, x, y, z, xs, ys, zs, cost):
    self.best_x = x
    self.best_y = y
    self.best_z = z

    self.best_xs = xs
    self.best_ys = ys
    self.best_zs = zs

    self.best_cost = cost

  def CreatePath(self, start, end):
    x = np.random.uniform(start[0], end[0], self.handles+2)
    x[0] = start[0]
    x[-1] = end[0]

    y = np.random.uniform(start[1], end[1], self.handles+2)
    y[0] = start[1]
    y[-1] = end[1]

    # z = np.random.uniform(start[2], end[2], self.handles+2)
    # z[0] = start[2]
    # z[-1] = end[2]
    z = np.linspace(start[2], end[2], self.handles+2)
    
    return x,y,z

class GlobalBest:
  def __init__(self, x, y, z, xs, ys, zs, cost):
    self.x = x
    self.y = y
    self.z = z
    self.xs = xs
    self.ys = ys
    self.zs = zs
    self.cost = cost

# Plotting 3d-Spheres
def ms(x, y, z, radius, resolution=10):
    u, v = np.mgrid[0:2*np.pi:resolution*2j, 0:np.pi:resolution*1j]
    X = radius * np.cos(u)*np.sin(v) + x
    Y = radius * np.sin(u)*np.sin(v) + y
    Z = radius * np.cos(v) + z
    return (X, Y, Z)

# Main
def PSO(origin, destination, geometry, no_particles, handles, iterations, min_limit, max_limit):

  # Create Particle Swarm
  particles = []
  for i in range(no_particles):
    particles.append(Particle(origin, destination, handles))

  # Initialize
  global_best = GlobalBest(0,0,0,0,0,0,1e6)
  for particle in particles:
    if particle.cost < global_best.cost:
      global_best.x = particle.x.copy()
      global_best.y = particle.y.copy()
      global_best.z = particle.z.copy()

      global_best.xs = particle.xs.copy()
      global_best.ys = particle.ys.copy()
      global_best.zs = particle.zs.copy()

      global_best.cost = particle.cost.copy()

  # Particle Swarm Optimization
  canvas_max_x = max_limit[0]
  canvas_min_x = min_limit[0]

  canvas_max_y = max_limit[1]
  canvas_min_y = min_limit[1]

  canvas_max_z = max_limit[2]
  canvas_min_z = min_limit[2]

  iterations = iterations
  w = 1 
  wdamp = .98
  c1 = 1.5
  c2 = 1.5
  alpha = .1
  vel_max_x = alpha*(canvas_max_x - canvas_min_x)
  vel_min_x = -vel_max_x
  vel_max_y = alpha*(canvas_max_y - canvas_min_y)
  vel_min_y = -vel_max_y
  vel_max_z = alpha*(canvas_max_z - canvas_min_z)
  vel_min_z = -vel_max_z

  for i in tqdm(range(0, iterations)):
    
    for particle in particles:

      # X
      particle.vx = w*particle.vx + \
                    c1*np.random.rand(1, handles+2).reshape(handles+2)*(particle.best_x-particle.x) + \
                    c2*np.random.rand(1, handles+2).reshape(handles+2)*(global_best.x-particle.x)
      
      particle.vx = np.maximum(particle.vx.copy(), vel_min_x)
      particle.vx = np.minimum(particle.vx.copy(), vel_max_x)
  
      particle.x = np.add(particle.x, particle.vx, out=particle.x, casting='unsafe')

      # Y
      particle.vy = w*particle.vy + \
                    c1*np.random.rand(1, handles+2).reshape(handles+2)*(particle.best_y-particle.y) + \
                    c2*np.random.rand(1, handles+2).reshape(handles+2)*(global_best.y-particle.y)
      
      particle.vy = np.maximum(particle.vy.copy(), vel_min_y)
      particle.vy = np.minimum(particle.vy.copy(), vel_max_y)

      particle.y = np.add(particle.y, particle.vy, out=particle.y, casting='unsafe')

      # Z
      particle.vz = w*particle.vz + \
                    c1*np.random.rand(1, handles+2).reshape(handles+2)*(particle.best_z-particle.z) + \
                    c2*np.random.rand(1, handles+2).reshape(handles+2)*(global_best.z-particle.z)
      
      particle.vz = np.maximum(particle.vz.copy(), vel_min_z)
      particle.vz = np.minimum(particle.vz.copy(), vel_max_z)

      particle.z = np.add(particle.z, particle.vz, out=particle.z, casting='unsafe')

      # Calculate cost
      particle.Cost()
      if particle.cost < particle.best_cost:
        particle.best_x = particle.x.copy()
        particle.best_y = particle.y.copy()
        particle.best_z = particle.z.copy()

        particle.best_xs = particle.xs.copy()
        particle.best_ys = particle.ys.copy()
        particle.best_zs = particle.zs.copy()

        particle.best_cost = particle.cost.copy()

      if particle.cost < global_best.cost:
        global_best.x = particle.x.copy()
        global_best.y = particle.y.copy()
        global_best.z = particle.z.copy()

        global_best.xs = particle.xs.copy()
        global_best.ys = particle.ys.copy()
        global_best.zs = particle.zs.copy()

        global_best.cost = particle.cost.copy()

    w = w * wdamp

  print(f'\nFinal Global Best Cost: {global_best.cost}')
  return global_best.xs, global_best.ys, global_best.zs, global_best.cost


Define CDR

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import interp1d

class Drone:
  def __init__(self, speed):
    self.speed = speed
  
  def get_track(self, x, y, z):
    self.x = x
    self.y = y
    self.z = z

    length = 0
    for i in range(0, len(x)):
      try:
        dx = x[i+1]-x[i]
        dy = y[i+1]-y[i]
        dz = z[i+1]-z[i]
        dh = np.sqrt(dx**2+dy**2+dz**2)
        length += dh
      except:
        break
    length = round(length,1)

    time = length/self.speed
    time_vector = np.arange(0, round(time,2)*10 + 1)/10
    xnew = np.linspace(x[0],x[-1],len(time_vector))
    interp = interp1d(x, y)
    yinterp = interp(xnew)
    znew = np.linspace(z[0],z[-1],len(time_vector))
    
    df = pd.DataFrame({'time':time_vector, 'x':xnew, 'y':yinterp, 'z':znew})
    df.x = df.x.round(2)
    df.y = df.y.round(2)
    df.z = df.z.round(2)
    df['pos'] = df[['x','y','z']].apply(tuple, axis=1)

    return df.drop(columns=['x', 'y', 'z'])

  def update_speed(self, speed):
    self.speed = speed
    return self.get_track(self.x, self.y, self.z)

class CDR():
  def __init__(self):
    self.master = []
    self.drones = []
    self.drone_index = 0

  def add_drone(self, drone, track):
    if len(self.master) == 0:
      self.master = drone.get_track(track[0], track[1], track[2])
      self.master.columns = ['time', f'drone_{self.drone_index}']
      self.drones.append(drone)
      self.drone_index += 1
    else:
      self.master = pd.concat([self.master.set_index('time'), drone.get_track(track[0], track[1], track[2]).set_index('time')], axis=1).reset_index()
      self.master.columns = [*self.master.columns[:-1], f'drone_{self.drone_index}']
      self.drones.append(drone)
      self.clean()
      self.drone_index += 1    

  def update_speed(self, drone, to_speed):
    self.master = pd.concat([self.master.set_index('time'), self.drones[drone].update_speed(to_speed).set_index('time')], axis=1).reset_index()
    self.master = self.master.drop(columns=[f'drone_{drone}'])
    self.master.columns = [*self.master.columns[:-1], f'drone_{drone}']
    self.master = self.master.reindex(sorted(self.master.columns), axis=1)
    time_column = self.master.pop('time')
    self.master.insert(0, 'time', time_column)
    self.clean()

  def update_delay(self, drone, units):
    self.master = self.master.reindex(range(len(self.master)+units))
    self.master[f'drone_{drone}'] = self.master[f'drone_{drone}'].shift(units)
    self.clean()
    self.master['time'] = np.arange(0, len(self.master))/10

  def clean(self):
    self.master = self.master.ffill(axis=0)
    self.master = self.master.bfill(axis=0)


Coordinator

In [12]:
origins = [(-5,-5,.5), (-5,-5,.5), (5,-5,.5), (5,-5,.5)]
destinations = [(5,5,.5), (5,5,.5), (-5,5,.5), (-5,5,.5)]

max_distance = .5
n = len(origins)

n_origins = []
n_destinations = []
for i in range(0, n):
  for j in range(i+1, n):
    # print(f'{i},{j}')
    dist_origin = np.sqrt((origins[i][0]-origins[j][0])**2+(origins[i][1]-origins[j][1])**2+(origins[i][2]-origins[j][2])**2)
    dist_destination = np.sqrt((destinations[i][0]-destinations[j][0])**2+(destinations[i][1]-destinations[j][1])**2+(destinations[i][2]-destinations[j][2])**2)
    
    if dist_origin <= max_distance and dist_destination <= max_distance:
      n_origins.append(tuple(np.add(origins[i], origins[j])/2))
      n_destinations.append(tuple(np.add(destinations[i], destinations[j])/2))
    else:
      n_origins.append(origins[i])
      n_destinations.append(destinations[i])

    # print(f'Distance Origin: {dist_origin}')
    # print(f'Distance Destination: {dist_destination}')

n_origins = pd.unique(n_origins)
n_destinations = pd.unique(n_destinations)

no_drones = []
for k in range(0, len(n_origins)):
  cumsum = 0
  for m in range(0, len(origins)):
    d = np.sqrt((n_origins[k][0]-origins[m][0])**2+(n_origins[k][1]-origins[m][1])**2+(n_origins[k][2]-origins[m][2])**2)
    if d <= max_distance: cumsum += 1
  no_drones.append(cumsum)

In [4]:
n_origins

array([(-5.0, -5.0, 0.5), (5.0, -5.0, 0.5)], dtype=object)

In [15]:
no_drones

[2, 2]

PSO

In [16]:
# Create model obstacles
bounds_x = (-10, 10)
bounds_y = (-10, 10)
bounds_z = (0, 3)

obstacles = [Obstacle(2,2,0,1.5), \
             Obstacle(-2,-2,0,1.5), \
             Obstacle(2,-2,0,1.0), \
             Obstacle(-2,2,0,1.0)]

# Create CDR
cdr = CDR()

# Run PSO
data = []
colors = ['red', 'blue', 'green']
for i in range(0, len(n_origins)):
  x, y, z, cost = PSO(origin=n_origins[i], destination=n_destinations[i], geometry=obstacles, no_particles=100, \
                    handles=3, iterations=100, min_limit=(bounds_x[0],bounds_y[0],bounds_z[0]), max_limit=(bounds_x[1],bounds_y[1],bounds_z[1]))

  # Append best line
  data.append(go.Scatter3d(x=x, y=y, z=z, mode='lines', name=f'Corridor {i}', line=dict(width=5, color=colors[i])))
  
  # for j in range(0, len(no_drones)):
  for k in range(0, no_drones[i]):
    cdr.add_drone(drone=Drone(speed=5), track=(x,y,z))

# Append 3d-spheres
for j in range(0, len(obstacles)):
    (x_pns_surface, y_pns_surface, z_pns_surface) = ms(obstacles[j].x, obstacles[j].y, obstacles[j].z, obstacles[j].r)
    data.append(go.Surface(x=x_pns_surface, y=y_pns_surface, z=z_pns_surface, showscale=False, opacity=1))

# Plot first frame of drones
data.append(go.Scatter3d(
            x=[cdr.master[f'drone_{m}'][0][0] for m in range(0,len(origins))],
            y=[cdr.master[f'drone_{m}'][0][1] for m in range(0,len(origins))],
            z=[cdr.master[f'drone_{m}'][0][2] for m in range(0,len(origins))],
            name='Drone',
            mode="markers",
            marker=dict(color="black", size=5)))

100%|██████████| 100/100 [00:14<00:00,  6.89it/s]



Final Global Best Cost: 146.71324650130114


100%|██████████| 100/100 [00:14<00:00,  6.90it/s]


Final Global Best Cost: 142.06061516370292


Plot

In [20]:
layout = go.Layout(scene=dict(
  aspectmode='manual',
  aspectratio=dict(x=1, y=(bounds_y[1]-bounds_y[0])/(bounds_x[1]-bounds_x[0]), z=(bounds_z[1]-bounds_z[0])/(bounds_x[1]-bounds_x[0])),
  bgcolor='rgba(0,0,0,0)',
  xaxis = dict(range=[bounds_x[0], bounds_x[1]], showgrid=True, gridcolor='rgba(0, 0, 0, .1)', backgroundcolor="rgba(0, 0, 0, 0)", showticklabels=True),
  yaxis = dict(range=[bounds_y[0], bounds_y[1]], showgrid=True, gridcolor='rgba(0, 0, 0, .1)', backgroundcolor="rgba(0, 0, 0, 0)", showticklabels=True),
  zaxis = dict(range=[bounds_z[0], bounds_z[1]], showgrid=True, gridcolor='rgba(0, 0, 0, .1)', backgroundcolor="rgba(0, 0, 0, 0)", showticklabels=True)),
  updatemenus=[dict(type="buttons", buttons=[dict(label="Play", method="animate",args=[None, {'frame':{'duration':1}}])])]
  )

frames=[go.Frame(
        data=[go.Scatter3d(
            x=[cdr.master[f'drone_{m}'][k][0] for m in range(0,len(origins))],
            y=[cdr.master[f'drone_{m}'][k][1] for m in range(0,len(origins))],
            z=[cdr.master[f'drone_{m}'][k][2] for m in range(0,len(origins))],
            name='Drone',
            mode="markers",
            marker=dict(color="black", size=5))], traces = [len(data)-1])

        for k in range(len(cdr.master.drone_1))]

fig = go.Figure(data=data, layout=layout, frames=frames)

camera = dict(
    up=dict(x=0, y=0, z=.1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0, y=-2, z=3)
)

fig.update_layout(scene_camera=camera)

fig.show()

In [19]:
cdr.update_speed(drone=0, to_speed=5)
cdr.update_speed(drone=1, to_speed=5)

cdr.update_speed(drone=2, to_speed=5)
cdr.update_speed(drone=3, to_speed=5)

cdr.update_delay(drone=1, units=2)

cdr.update_delay(drone=2, units=4)
cdr.update_delay(drone=3, units=6)

In [18]:
cdr.master

,time,drone_0,drone_1,drone_2,drone_3
0,0.0,"(-5.0, -5.0, 0.5)","(-5.0, -5.0, 0.5)","(-5.0, 5.0, 0.5)","(-5.0, 5.0, 0.5)"
1,0.1,"(-4.67, -4.25, 0.5)","(-4.67, -4.25, 0.5)","(-4.66, 4.39, 0.5)","(-4.66, 4.39, 0.5)"
2,0.2,"(-4.33, -3.49, 0.5)","(-4.33, -3.49, 0.5)","(-4.31, 3.83, 0.5)","(-4.31, 3.83, 0.5)"
3,0.3,"(-4.0, -2.76, 0.5)","(-4.0, -2.76, 0.5)","(-3.97, 3.29, 0.5)","(-3.97, 3.29, 0.5)"
4,0.4,"(-3.67, -2.08, 0.5)","(-3.67, -2.08, 0.5)","(-3.62, 2.77, 0.5)","(-3.62, 2.77, 0.5)"
5,0.5,"(-3.33, -1.48, 0.5)","(-3.33, -1.48, 0.5)","(-3.28, 2.28, 0.5)","(-3.28, 2.28, 0.5)"
6,0.6,"(-3.0, -0.94, 0.5)","(-3.0, -0.94, 0.5)","(-2.93, 1.8, 0.5)","(-2.93, 1.8, 0.5)"
7,0.7,"(-2.67, -0.46, 0.5)","(-2.67, -0.46, 0.5)","(-2.59, 1.35, 0.5)","(-2.59, 1.35, 0.5)"
8,0.8,"(-2.33, -0.02, 0.5)","(-2.33, -0.02, 0.5)","(-2.24, 0.91, 0.5)","(-2.24, 0.91, 0.5)"
9,0.9,"(-2.0, 0.37, 0.5)","(-2.0, 0.37, 0.5)","(-1.9, 0.5, 0.5)","(-1.9, 0.5, 0.5)"
